In [ ]:
# TODO: training code

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence

import numpy as np

import utils.preprocess_text import Dictionary
from model import Encoder, Decoder
from dataloader import get_loader
from torchvision import transforms

SyntaxError: invalid syntax (<ipython-input-1-e94e96f084f7>, line 7)